# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import numpy as np #импортируем для работы с данными
import pandas as pd #импорт библиотеки Pandas
from sklearn.model_selection import train_test_split #для разделения выборки
from sklearn.linear_model import LinearRegression #логистическая регрессия
from sklearn.metrics import mean_squared_error #для нахождения RSE
from sklearn.preprocessing import MinMaxScaler #для стандартизации
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
data_1 = pd.read_csv("/datasets/geo_data_0.csv") #загрузка датасетов
data_2 = pd.read_csv("/datasets/geo_data_1.csv")
data_3 = pd.read_csv("/datasets/geo_data_2.csv")

In [3]:
def chek(data): #функция для проверки на дубликаты и т.п.
    print("Количество пропусков :", data.isna().sum().sum()) #сумма пропусков
    print("Количество дубликатов :", data.duplicated().sum())
    print("Информация о датасете :") #информация о датасете
    data.info()
    print(data.head(5)) #первые 5 строк

In [4]:
chek(data_1) #информация о первом датасете

Количество пропусков : 0
Количество дубликатов : 0
Информация о датасете :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


In [5]:
chek(data_2) #втором

Количество пропусков : 0
Количество дубликатов : 0
Информация о датасете :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


In [6]:
chek(data_3) #третьем

Количество пропусков : 0
Количество дубликатов : 0
Информация о датасете :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


Как мы видим, с данными все впорядке, предлагаю удалить столбец id, так как он не нужен в обучении.

In [7]:
data_1 = data_1.drop(["id"], axis = 1) #удаляем столбцы
data_2 = data_2.drop(["id"], axis = 1)
data_3 = data_3.drop(["id"], axis = 1)

Таким образом, данные готовы для дальнейшей работы, был произведен первчиный анализ, в результате чего:
* f1, f2, f3 - признаки
* product - целевой признак

## Обучение и проверка модели

In [8]:
def split_train(data):
    features = data.drop(["product"], axis = 1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 12345)
    
    scaler = StandardScaler() #стандартизация данных
    scaler.fit(features_train) #вычисляем среднее
    features_train = scaler.transform(features_train) #преобразуем обучающую и валидационную выборки
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression() #создаем модель
    model.fit(features_train, target_train) #обучаем ее на тренировочных данных
    predictions_valid = model.predict(features_valid) # предсказание на валидационных данных
    predictions_valid = pd.Series(predictions_valid) #преобразуем в Series
    rmse = (mean_squared_error(predictions_valid, target_valid))**(0.5) #находим RMSE
    mean_product = sum(predictions_valid) / len(predictions_valid) #средний запас сырья
    print("RMSE:" ,rmse) #вывод RMSE
    print("Средний запас:",mean_product) #Средний запас
    return (predictions_valid, target_valid.reset_index(drop=True), rmse)  #Возвращаем данные для дальнейшей работы 

0 скважина

In [9]:
split_train(data_1)

RMSE: 37.5794217150813
Средний запас: 92.59256778438008


(0         95.894952
 1         77.572583
 2         77.892640
 3         90.175134
 4         70.510088
             ...    
 24995    103.037104
 24996     85.403255
 24997     61.509833
 24998    118.180397
 24999    118.169392
 Length: 25000, dtype: float64,
 0         10.038645
 1        114.551489
 2        132.603635
 3        169.072125
 4        122.325180
             ...    
 24995    170.116726
 24996     93.632175
 24997    127.352259
 24998     99.782700
 24999    177.821022
 Name: product, Length: 25000, dtype: float64,
 37.5794217150813)

1 скважина

In [10]:
split_train(data_2)

RMSE: 0.893099286775617
Средний запас: 68.7285468954458


(0         82.663314
 1         54.431786
 2         29.748760
 3         53.552133
 4          1.243856
             ...    
 24995    136.869211
 24996    110.693465
 24997    137.879341
 24998     83.761966
 24999     53.958466
 Length: 25000, dtype: float64,
 0         80.859783
 1         53.906522
 2         30.132364
 3         53.906522
 4          0.000000
             ...    
 24995    137.945408
 24996    110.992147
 24997    137.945408
 24998     84.038886
 24999     53.906522
 Name: product, Length: 25000, dtype: float64,
 0.893099286775617)

2 скважина

In [11]:
split_train(data_3)

RMSE: 40.02970873393434
Средний запас: 94.96504596800506


(0         93.599633
 1         75.105159
 2         90.066809
 3        105.162375
 4        115.303310
             ...    
 24995     78.765887
 24996     95.603394
 24997     99.407281
 24998     77.779912
 24999    129.032417
 Length: 25000, dtype: float64,
 0         61.212375
 1         41.850118
 2         57.776581
 3        100.053761
 4        109.897122
             ...    
 24995     28.492402
 24996     21.431303
 24997    125.487229
 24998     99.422903
 24999    127.445075
 Name: product, Length: 25000, dtype: float64,
 40.02970873393434)

In [12]:
predictions_valid_0, target_valid_0, rmse_0 = split_train(data_1) #сохраняем данные
predictions_valid_1, target_valid_1, rmse_1 = split_train(data_2)
predictions_valid_2, target_valid_2, rmse_2 = split_train(data_3)

RMSE: 37.5794217150813
Средний запас: 92.59256778438008
RMSE: 0.893099286775617
Средний запас: 68.7285468954458
RMSE: 40.02970873393434
Средний запас: 94.96504596800506


Таким образом:
* скважина 2 - самый высокий запас нефти, однако rmse высокий, что свидетельсвует о неточности обучения линейной регрессии
* скважина 0 - сравнимые с первой скважиной параметры
* скважина 1 - средний запас меньше чем, у других, но rmse низкий, что означает о хорошей обучаемости модели.

## Подготовка к расчёту прибыли

Условия задачи:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [13]:
budjet = 10**10 #бюджет
tocka_top = 200 #лучшее количество точек
tocka_total = 500 #общее количество точек
price = 450000 #доход на единицу продукта

Чтобы рассчитать среднее количество продуктов, нужно бюджет разделить на доход от лучших точек.

In [14]:
total_price = budjet / (tocka_top * price) #выводим формулу
total_price

111.11111111111111

Таким образом, чтобы окупиться нужно в среднем 111 единиц продукции. Данное значение превышает все имеющиеся скважины.

## Расчёт прибыли и рисков 

In [15]:
def price_delta(predictions, target): #функция для расчета прибыли для 200 скважин
    top_predictions = predictions.sort_values(ascending = False)
    top_target = target[top_predictions.index][:200] # лучшее 200
    profit  =  top_target.sum() * price #прибыль с реальных точек
    return profit - budjet 

In [20]:
state = np.random.RandomState(12345) #случайный шаг
def bootstrap_1000(predictions, target): #функция для бутстрапа
    profit=[] #пустой лист
    lost = 0 #счетчик
    for i in range(1000): #доверительный интервал
        target_sample = target.sample(replace=True, random_state=state, n=500)
        predictions_sample = predictions[target_sample.index]
        profit.append(price_delta(predictions_sample, target_sample))
    profit = pd.Series(profit)
    lower = profit.quantile(0.025) #квантиль нижний
    upper = profit.quantile(0.975) #квантиль верхний
    mean = profit.mean() #средняя выручка
    risk = (profit < 0).mean() #риск
    print("Средняя выручка :", mean)
    print(f'Доверительный интервал от {lower} до {upper}')
    print("Риск",risk)
    return mean, lower, upper, risk


Для Первого региона

In [21]:
bootstrap_1000(predictions_valid_0, target_valid_0)

Средняя выручка : 425938526.91059244
Доверительный интервал от -102090094.83793654 до 947976353.3583689
Риск 0.06


(425938526.91059244, -102090094.83793654, 947976353.3583689, 0.06)

Для Второго региона

In [22]:
bootstrap_1000(predictions_valid_1, target_valid_1)

Средняя выручка : 518259493.6973249
Доверительный интервал от 128123231.43308444 до 953612982.0669085
Риск 0.003


(518259493.6973249, 128123231.43308444, 953612982.0669085, 0.003)

Для Третьего региона

In [19]:
bootstrap_1000(predictions_valid_2, target_valid_2)

Средняя выручка : 420194005.3440501
Доверительный интервал от -115852609.16001143 до 989629939.8445739
Риск 0.062


(420194005.3440501, -115852609.16001143, 989629939.8445739, 0.062)

Таким образом, отсеиваем первый и третий варианты, т.к их риск выше нежели у второго региона.
Нижняя граница доверительного интервала больше минимальной окупаемости следовательно - это оптимальное решение.

## Чек-лист готовности проекта

## Итог

Таким образом, в рамках работы были изучены данные, проведена подгтовка, обучена линейная регрессия для каждого региона, расчитана функция расчета прибыли и рисков для данных регионов. Было принято решение, использовать вторую скважину для дальнешей работы.
